<a href="https://colab.research.google.com/github/Gus-oss/MachineLearning/blob/main/MachineLearningTarea5_Agrupamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Learning: Análisis y técnicas de machine learning de los datos sobre influenza, COVID-19 y otros virus respiratorios.

## Paquetes por instalar

In [ ]:
!pip install pandas scikit-learn gower

## Librerias y creación de carpeta de imagenes.

In [ ]:
#Importamos las librerias que utilizaremos a lo largo del documento
#import pandas as pd
import numpy as np
import statistics as stats   #utilizado en la parte estadistica descriptiva
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
import os #Se utilizo para guardar las imagenes (carpeta) y para guardar las imagenes en la carpeta
import plotly.graph_objects as go #esta libreria es para hacer graficos dinamicos
from statsmodels.graphics.gofplots import qqplot #se utilizo para hacer la grafica cuartil cuartil

# Librerias para Machine Learning
from sklearn.feature_selection import f_regression #para la seleccion de caracteristicas
from sklearn.preprocessing import MinMaxScaler #para tener todas las variables entre 0 y 1
from sklearn.feature_selection import VarianceThreshold #para la prueba de varianza
from sklearn.feature_selection import mutual_info_regression #para la prueba de seleccion mutua
from sklearn.linear_model import LinearRegression #regresion lineal simple
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS #Hace toda la combinatoria
#de las variables que tenemos en un conjunto de datos
from mlxtend.feature_selection import SequentialFeatureSelector as SFS #para la parte de sequential selector
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN #para agrupación
import gower #para dbscan con la metrica de gower
from sklearn.neighbors import NearestNeighbors# para la grafica del codo
from sklearn.metrics import silhouette_score #grafica del codo
from sklearn.manifold import TSNE #vizualizacion de cluster
from sklearn.decomposition import PCA #vizualizacion de cluster

# Carpeta única para guardar las imágenes
IMAGES_PATH = "Galeria"  # Nombre de la carpeta (puedes cambiarlo)
os.makedirs(IMAGES_PATH, exist_ok=True) # Crea la carpeta si no existe

def save_fig(fig_id, tight_layout=True, fig_extension="pdf", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Imagen guardada como ", fig_id, "en la carpeta", IMAGES_PATH) #Mensaje mas informativo
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

#Recomendación del Dr. Alberto Benavidez es generar imagenes en PDF para la creacion de articulos.
#A demas de antes de llamar la función, escribir plt.tight_layout() para quitar espacios
#inecesarios en las imagenes, este plt.tight_layout() ya se encuentra en la funcion pra guardar imagenes

## Obtencion de datos.

Los datos se actualizan cada cierto periodo de tiempo. Para esto, se automatizo la optención de los datos para que al momento de ejecutar el programa se estudien los datos mas recientes posibles publicados por la Direccion de Epidemiología de México.

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL de la página de datos abiertos
url = 'https://www.gob.mx/salud/documentos/datos-abiertos-152127'

# Obtener el HTML de la página
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')

# Imprimir todos los enlaces en la página para ver su estructura
print("Enlaces encontrados en la página:")
for a in soup.find_all('a', href=True):
    print(a['href'])

Enlaces encontrados en la página:
/
/tramites
/gobierno
/salud/en
https://www.gob.mx/busqueda?utf8=✓
?tab=
/salud/es/archivo/prensa
/salud/es/#390
/salud/es/archivo/documentos
/salud/es/archivo/acciones_y_programas
/salud/es/archivo/multimedia
http://portal.salud.gob.mx/sites/transparencia/transparencia/proteccion_datos/proteccion_1_1.html
http://portal.salud.gob.mx/sites/transparencia/transparencia/transparencia/transparencia_1.html
http://www.gob.mx
/salud
/salud/articulos/lavado-de-manos-esencial-en-la-prevencion-de-infecciones
/salud/articulos/hospital-general-de-mexico-realiza-jornada-de-reconstruccion-mamaria-en-beneficio-de-mujeres-sobrevivientes-de-cancer
/salud/articulos/secretaria-de-salud-reconoce-en-su-dia-a-sus-trabajadoras-y-trabajadores-por-su-esfuerzo-y-compromiso-con-el-pais
/salud/articulos/gana-archivo-historico-de-la-secretaria-de-salud-premio-por-rescatar-y-conservar-documentos-relevantes-para-la-historia-de-mexico
/salud/articulos/hospital-general-de-mexico-conmem

In [ ]:
import requests
import zipfile
import io

# URL del Diccionario de datos
diccionario_url = 'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_influenza_covid19.zip'

# Descargar y abrir el archivo ZIP del Diccionario de datos
diccionario_zip_response = requests.get(diccionario_url)
diccionario_zip_response.raise_for_status()

with zipfile.ZipFile(io.BytesIO(diccionario_zip_response.content)) as diccionario_zip:
    # Listar los nombres de los archivos en el ZIP
    print("Archivos contenidos en el ZIP del Diccionario de datos:")
    for file_name in diccionario_zip.namelist():
        print(file_name)

ConnectionError: HTTPSConnectionPool(host='datosabiertos.salud.gob.mx', port=443): Max retries exceeded with url: /gobmx/salud/datos_abiertos/datos_abiertos_influenza_covid19.zip (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7931f410c2c0>: Failed to resolve 'datosabiertos.salud.gob.mx' ([Errno -3] Temporary failure in name resolution)"))

In [ ]:
import requests
import zipfile
import io
import pandas as pd

# URLs directas para los archivos ZIP
covid_data_url = 'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_influenza_covid19.zip'
diccionario_url = 'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_abiertos.zip'

def descargar_datos_covid():
    # Descargar y procesar el archivo ZIP de datos COVID-19
    print("Descargando archivo ZIP de datos COVID-19 desde:", covid_data_url)
    covid_zip_response = requests.get(covid_data_url)
    covid_zip_response.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(covid_zip_response.content)) as covid_zip:
        for file_name in covid_zip.namelist():
            if file_name.endswith('.csv'):
                print(f"Extrayendo {file_name}...")
                with covid_zip.open(file_name) as csv_file:
                    df_covid = pd.read_csv(csv_file)
                    print("Datos COVID-19 cargados correctamente en el DataFrame.")
                    break  # Salir después de encontrar el archivo CSV

    # Descargar y procesar el archivo ZIP del Diccionario de datos
    print("Descargando archivo ZIP del Diccionario de datos desde:", diccionario_url)
    diccionario_zip_response = requests.get(diccionario_url)
    diccionario_zip_response.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(diccionario_zip_response.content)) as diccionario_zip:
        # Cargar los archivos Excel
        excel_file_catalogos = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'))
        df_descriptores = pd.read_excel(diccionario_zip.open('240708 Descriptores_.xlsx'))
        print("Archivos del Diccionario de datos cargados correctamente en DataFrames.")

        # Cargar hojas específicas en DataFrames separados
        df_enti = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'), sheet_name='Catálogo de ENTIDADES')
        df_muni = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'), sheet_name='Catálogo MUNICIPIOS')
        df_sector = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'), sheet_name='Catálogo SECTOR')
        df_pcr = pd.read_excel(diccionario_zip.open('240708 Catalogos.xlsx'), sheet_name='Catálogo RESULTADO_PCR')

    return df_covid, excel_file_catalogos, df_descriptores, df_enti, df_muni, df_sector, df_pcr

# Ejecutar la función y cargar los datos en los DataFrames
df_covid, excel_file_catalogos, df_descriptores, df_enti, df_muni, df_sector, df_pcr = descargar_datos_covid()

In [ ]:
#df_covid = pd.read_csv('/content/COVID19MEXICO.csv')
#correr esta celda cuando el codigo de extraccion de datos automatico no funcione (que raro que siempre me pase en viernes)

## Limpieza de la base de datos.

En este apartado nos encargaremos de checar valores nulos, formato de fechas, filas duplicadas y verificaremos errores al momento de capturar datos.

### Vizualisación de la base de datos

In [ ]:
df_covid #visualizamos la base de datos

### Valores nulos

In [ ]:
# Revisión de valores faltantes (NaN)
# Verificamos cuántos valores faltantes hay por columna
valores_nulos = df_covid.isnull().sum()
print("Valores faltantes por columna:\n", valores_nulos)

Podemos encontrar que en el conjunto de datos no hay datos faltantes.

### Fortmato Fechas

In [ ]:
# Revisión de tipos de datos
# Convertimos las columnas de fecha a formato datetime para análisis temporal
df_covid['FECHA_INGRESO']  = pd.to_datetime(
    df_covid['FECHA_INGRESO'],
    format="%Y-%m-%d",
    errors="coerce"
)

df_covid['FECHA_SINTOMAS'] = pd.to_datetime(
    df_covid['FECHA_SINTOMAS'],
    format="%Y-%m-%d",
    errors="coerce"
)

df_covid['FECHA_DEF']      = pd.to_datetime(
    df_covid['FECHA_DEF'],
    format="%Y-%m-%d",
    errors="coerce"
)

In [ ]:
df_covid["FECHA_INGRESO"].unique() #NO CUENTA CON VALORES NaT

In [ ]:
df_covid["FECHA_SINTOMAS"].unique() #No cuenta con valores NaT

In [ ]:
df_covid["FECHA_DEF"].unique()

Al cambiar las fechas a datatime con el formato yyyy-mm-dd apreciamos que en la columna *FECHA_DEF* se encuentran valores NaT( conjunto original de datos se tenia como 9999-99-99, pero al hacer la conversion a datatime se colodaron NaT)  los cuales significan Not a Time, lo que quiere decir que no se tiene registro de fecha de defunción. Esto puede significar :


*   el paciente fallecio y no se registro la fecha de defunción
*   o el paciente no fallecio y se recupero

Se investiga cuantos NaT se tienen registrados en el conjunto de datos.

In [ ]:
# Contar los valores NaT en una columna específica
nat_count_fecha_def = df_covid['FECHA_DEF'].isna().sum()
print(f"Valores NaT en la columna FECHA_DEF: {nat_count_fecha_def}")

In [ ]:
# Obtener el número de filas
num_filas = df_covid.shape[0]
print(f"El DataFrame tiene {num_filas} filas.")

Como el numero de NaT que hay en la columna FECHA_DEF es muy cercano al numero de filas que tiene el dataframe df_covid se opto por no eliminar las filas que tengan NaT y se conservaran.

### Duplicados

In [ ]:
num_filas_original = df_covid.shape[0]
num_filas_unicas = df_covid.drop_duplicates().shape[0]
print(f"Total filas originales: {num_filas_original}")
print(f"Total filas únicas: {num_filas_unicas}")
#En caso de encontrar filas duplicadas se tendria que utilizar df_covid = df_covid.drop_duplicates()

No se encontraron filas duplicadas en el conjunto de datos.

**texto en negrita**### Edad

In [ ]:
df_covid['EDAD'].unique()

Verificamos que las edades estan correctas y que no hay errores de typing al momento de realizar la captura de las edades.

## Estadistica Descriptiva.

En esta sección realizaremos:

- Comprobar si en el conjunto de datos son parametricos o no parametricos.
- Calcular estadisticos descriptivos básicos.
- Matriz de correlaciones.
- Realizar pruebas de hipótesis a partir de las conclusiones obtenidas en la matriz de correlación.
- Realizar graficos estadisticos.

### Estadisticos.

Realizamos un analisis de los estadisticos de media, moda y mediana a la variable EDAD del conjunto de datos.

In [ ]:
edades = df_covid['EDAD'].tolist()

print("Promedio:", stats.mean(edades))
print("Mediana :", stats.median(edades))
print("Moda(s) :", stats.multimode(edades))

La mayoría de los casos de COVID-19 en este conjunto de datos corresponden a adultos jóvenes (aproximadamente 37 años, mediana 35 años). Sin embargo, la moda de 1 año sugiere un pico artificial o un sesgo: podría reflejar una alta proporción de casos en menores de 1 año.

Ahora realizamos un diagrama de cajas con vigotes para esta misma variable

In [ ]:
# Elimina valores nulos si existen
edades = df_covid['EDAD']

plt.figure(figsize=(6, 8))
plt.boxplot(edades, vert=False)
plt.title('Distribución de la edad de pacientes COVID-19')
plt.ylabel('Edad (años)')
plt.grid(axis='y', linestyle='--', alpha=0.7)  # líneas de referencia
plt.show()


save_fig("boxplot_edad", fig_extension="pdf")

In [ ]:
plt.figure(figsize=(8,6))
sns.violinplot(x=df_covid['EDAD'], color="skyblue", inner="quartile")
plt.axvline(
    x = stats.quantiles(df_covid['EDAD'], n=4)[0],
    label = '$Q_1$',
    linestyle = '--',
    color = 'green'
)
plt.axvline(
    x = stats.quantiles(df_covid['EDAD'], n=4)[1],
    label = '$Q_2$',
    linestyle = '-',
    color = 'blue'
)
plt.axvline(
    x = stats.quantiles(df_covid['EDAD'], n=4)[2],
    label = '$Q_3$',
    linestyle = '--',
    color = 'green'
)
plt.legend()
plt.title("Distribución de la edad de pacientes COVID-19")
plt.xlabel("Edad (años)")

# Guardar la figura
save_fig("violinplot_edad", fig_extension="pdf")

plt.show()

Grafica Cuartil-cuartil de las edades del conjunto de datos del covid-19

In [ ]:
plt.figure()
qqplot(df_covid['EDAD'], line ='s')

# Guardar la figura
save_fig("cuartilcuartil_edad", fig_extension="pdf")
plt.show()

De acuerdo a la grafica, se sospecha que la muestra respecto a las edades no es parametrica.

### Correlaciones

In [ ]:
df_covid.columns

In [ ]:
columns_to_replace = [
    'NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
    'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA'
]

for col in columns_to_replace:
    df_covid[col] = df_covid[col].replace({'SI': 1}).astype('Int64')  # Usar 'Int64' para manejar valores nulos


# Filtrar pacientes intubados
df_covid19_intubado = df_covid[df_covid['INTUBADO'] == 1]  # Usamos 1 para "Sí"

# Seleccionar las columnas de interés
df_covid19_resultado = df_covid19_intubado[['OBESIDAD', 'NEUMONIA', 'EDAD',
                                                'NACIONALIDAD', 'EMBARAZO', 'INDIGENA', 'DIABETES',
                                                'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
                                                'CARDIOVASCULAR', 'RENAL_CRONICA', 'TABAQUISMO', 'OTRO_CASO']]

# Reemplazar valores específicos por NaN
df_covid19V2_resultado = df_covid19_resultado.replace(['Se ignora', 'No Aplica', 'No especificado'], np.nan)

# Eliminar filas con valores NaN
df_covid19V2_resultado_clean = df_covid19_resultado.dropna()

# Crear el heatmap de correlación
plt.figure(figsize=(12, 8))
sns.heatmap(df_covid19V2_resultado_clean.corr(), annot=True,annot_kws={"size": 8}, fmt=".2f", cmap="coolwarm", linewidths=.5)
plt.title('Correlación de Condiciones en Pacientes Intubados de virus respiratorios')
save_fig('Correlacion_Condiciones_Pacientes_Intubados_COVID19' , fig_extension="pdf")
plt.show()

### Pruebas de hipotesis

In [ ]:
#Primera prueba de hipotesis

## Selección de Caracteristicas

En este apartado realizaremos:

- Aplicar algún método de filtro a tus datos mediante el uso de SelectKBest
- Aplicar los modelos de selección de características cuidando los supuestos de cada modelo
- Busca una o varias métricas para seleccionar características en literatura relacionada del problema.
- Con base en tu investigación, determina las características más relevantes de tu conjunto de datos.

### Separacion de las columnas.

Nos interesara saber si un paciente da positivo a SARS-CoV-2, por ese motivo, elegimos la variable dependiente



```
y = df_covid[["CLASIFICACION_FINAL_COVID"]]
```

Mientras que las demas variables del conjunto de datos seran nuestras variables independientes

```
x = df_covid[[ 'EDAD', 'NEUMONIA',
              'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA',
              'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO']]
```


Encontramos si hay linealidad entre las columnas independientes y dependientes

In [ ]:
df_covid_dropna = df_covid.dropna()
y = df_covid_dropna[["CLASIFICACION_FINAL_COVID"]]
x = df_covid_dropna[[ 'EDAD', 'NEUMONIA','EMBARAZO', 'DIABETES', 'EPOC', 'ASMA',
                      'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO']]

In [ ]:
valor_f = f_regression(x,y)
valor_f

In [ ]:
#Se crea un arreglo para identificar cuales variables pasan la prueba de linealidad
pasan_prueba = []
no_pasan_prueba=[]
alpha = 0.05

for i in range (len(valor_f[0])):
  if valor_f[0][i] < alpha:
    pasan_prueba.append(x.columns[i])
  else:
    no_pasan_prueba.append(x.columns[i])

In [ ]:
pasan_prueba

In [ ]:
df_resultados = pd.DataFrame(valor_f[0], index = x.columns, columns= ['valor_f']).sort_values('valor_f', ascending = True)
df_resultados

In [ ]:
plt.figure()
plt.bar(df_resultados.index, df_resultados.valor_f)
plt.show()

### Umbral de varianza

Para esto, tenemos que normalizar las variables.

In [ ]:
escalador = MinMaxScaler()
escala = escalador.fit_transform(x)
x_escalada = pd.DataFrame(escala, columns=x.columns)
x_escalada

Una vez obtenida las variables normalizadas, obtenemos las varianzas

In [ ]:
selector = VarianceThreshold()
selector.fit_transform(x_escalada)
selector.variances_

In [ ]:
df_resultados['varianza'] = selector.variances_
df_resultados

In [ ]:
plt.figure()
plt.bar(df_resultados.index, df_resultados.varianza)
plt.show()

### Información mutua

Nos permite encontrar relacions no lineales entre los datos

In [ ]:
mi = mutual_info_regression(x,y)
mi

In [ ]:
df_resultados['mi'] = mi
df_resultados

In [ ]:
plt.figure()
plt.bar(df_resultados.index, df_resultados.mi)
plt.show()

Resultados de variables dependientes en manera no lineal

### Reresion lineal

In [ ]:
lr = LinearRegression()
lr

In [ ]:
efs = EFS(estimator = lr,   #use linear regression as the classifier/estimator
          min_features = 1, #the minimum number of features to consider is 1
          max_features = 5, # the maximum number of features to consider is 4
          scoring = 'neg_mean_absolute_error', #The metric to use to evluate the classifier is accuracy
          cv = 5)

In [ ]:
efs = efs.fit(x,y)

In [ ]:
efs.best_score_

In [ ]:
#la mejor combinacion de la tabla es
efs.best_feature_names_


In [ ]:
df_resultados_efs = pd.DataFrame(efs.get_metric_dict()).T
df_resultados_efs = df_resultados_efs.sort_values('avg_score', ascending = False)
df_resultados_efs

In [ ]:
plt.figure()

plt.plot(
    df_resultados_efs.iloc[:5].feature_names.astype(str),
    df_resultados_efs.iloc[:5].avg_score

)
#plt.gca().invert_yaxis()
plt.xticks(rotation = 90)
plt.plot()

queremos el mas cercano al cero para considerar la mejor

### Sequencial Feature Selector

In [ ]:
sfs = SFS(
    estimator = lr,
    k_features = (1,5),
    forward = True,
    scoring = 'neg_mean_absolute_error',
    cv=5
)

In [ ]:
sfs = sfs.fit(x,y)

In [ ]:
sfs.k_feature_names_

In [ ]:
pd.DataFrame(sfs.subsets_).T.sort_values('avg_score', ascending = False)

Las variables que mas contribuyen son la edad y el embarazo

In [ ]:
df_covid

In [ ]:
df_covid.columns

## Agrupación

### Filtrado

In [ ]:
#filtrado de las variables categoricas
# Lista de columnas seleccionadas
columnas_gow = [
 'SECTOR', 'ENTIDAD_UM', 'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES',
 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'NACIONALIDAD',
 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA', 'DIABETES', 'EPOC', 'ASMA',
 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
 'RENAL_CRONICA', 'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB',
 'RESULTADO_PCR', 'RESULTADO_PCR_COINFECCION', 'TOMA_MUESTRA_ANTIGENO',
 'RESULTADO_ANTIGENO', 'UCI'
]

# Filtramos el dataset original
df_gow = df_covid[columnas_gow + ['CLASIFICACION_FINAL_COVID']].sample(n=20000, random_state=42).copy()

#Seleccionamos una muestra del total de filas, debido a que  Gower aumenta al cuadrado de el numero de filas
#por ende no podre poner todas, gano la batalla pero no al guerra

### DBSCAN + Gower

In [ ]:
# Convertimos todas las columnas numéricas a float
df_gow = df_gow.astype(float)

# Calculamos la matriz de distancias de Gower
dist_gower = gower.gower_matrix(df_gow)

Determinamos eps y min_sample optimos para DBSCAN, de acuerdo a las buenas practicas de https://stataiml.com/posts/how_to_set_dbscan_paramter/

In [ ]:
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print(" BÚSQUEDA DE PARÁMETROS ÓPTIMOS PARA DBSCAN")
print("="*80)

#DETERMINAR MIN_SAMPLES BASADO EN DIMENSIONALIDAD
n_features = len(columnas_gow)  # 32 columnas
n_samples = len(df_gow)  # 20,000 filas

print(f"\n INFORMACIÓN DEL DATASET:")
print(f"   • Número de características (dimensiones): {n_features}")
print(f"   • Número de muestras: {n_samples:,}")

# Reglas para MinPts según la literatura:
min_pts_candidates = {
    'Regla 1 (= dimensionalidad)': n_features,
    'Regla 2 (2 * dimensionalidad)': 2 * n_features,
    'Regla 3 (conservador para ruido)': int(2.5 * n_features),
    'Regla 4 (muy conservador)': int(3 * n_features)
}

print(f"\n CANDIDATOS PARA MIN_SAMPLES:")
print("-" * 60)
for regla, valor in min_pts_candidates.items():
    print(f"   • {regla}: {valor}")

#FUNCIÓN PARA DETECTAR EL CODO (KNEE/ELBOW)
def detectar_codo(x, y):
    """
    Detecta el punto del codo usando el método de la máxima curvatura.
    """
    # Normalizamos los datos
    x_norm = (x - np.min(x)) / (np.max(x) - np.min(x))
    y_norm = (y - np.min(y)) / (np.max(y) - np.min(y))

    # Calculamos la línea que conecta el primer y último punto
    p1 = np.array([x_norm[0], y_norm[0]])
    p2 = np.array([x_norm[-1], y_norm[-1]])

    # Calculamos la distancia perpendicular de cada punto a esta línea
    distancias = []
    for i in range(len(x_norm)):
        p = np.array([x_norm[i], y_norm[i]])
        # Distancia del punto a la línea
        d = np.abs(np.cross(p2-p1, p1-p)) / np.linalg.norm(p2-p1)
        distancias.append(d)

    # El codo es donde la distancia es máxima
    knee_idx = np.argmax(distancias)

    return knee_idx, y[knee_idx]

# MÉTODO K-NN PARA ENCONTRAR EPS ÓPTIMO
print(f"\n\n CALCULANDO DISTANCIAS K-NN PARA CADA CANDIDATO...\n")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

resultados_eps = {}
idx = 0

for nombre_regla, k_value in min_pts_candidates.items():
    print(f"Procesando k = {k_value} ({nombre_regla})...")

    # Calculamos distancias al k-ésimo vecino más cercano
    k_distances = []

    for i in range(len(dist_gower)):
        # Obtenemos las distancias ordenadas (excluyendo la distancia a sí mismo)
        distances = np.sort(dist_gower[i])[1:k_value+1]
        # Guardamos la distancia al k-ésimo vecino
        k_distances.append(distances[-1])

    # Ordenamos las distancias
    k_distances_sorted = np.sort(k_distances)

    # Detectamos el codo automáticamente
    x_vals = np.arange(len(k_distances_sorted))
    knee_idx, eps_optimo = detectar_codo(x_vals, k_distances_sorted)

    # También calculamos percentiles como referencia
    eps_p90 = np.percentile(k_distances_sorted, 90)
    eps_p95 = np.percentile(k_distances_sorted, 95)
    eps_p98 = np.percentile(k_distances_sorted, 98)

    # Guardamos resultados
    resultados_eps[k_value] = {
        'nombre': nombre_regla,
        'eps_knee': eps_optimo,
        'eps_p90': eps_p90,
        'eps_p95': eps_p95,
        'eps_p98': eps_p98,
        'knee_idx': knee_idx,
        'k_distances': k_distances_sorted
    }

    # Graficamos
    axes[idx].plot(k_distances_sorted, linewidth=2, color='steelblue', label='Distancias k-NN')

    # Marcamos el codo detectado
    axes[idx].scatter([knee_idx], [eps_optimo], color='red', s=200, zorder=5,
                      marker='*', edgecolors='black', linewidths=2,
                      label=f'Codo detectado')
    axes[idx].axhline(y=eps_optimo, color='red', linestyle='--', linewidth=2,
                      label=f'Knee/Elbow: {eps_optimo:.4f}')
    axes[idx].axhline(y=eps_p90, color='orange', linestyle=':', alpha=0.7,
                      label=f'P90: {eps_p90:.4f}')
    axes[idx].axhline(y=eps_p95, color='green', linestyle=':', alpha=0.7,
                      label=f'P95: {eps_p95:.4f}')

    axes[idx].set_xlabel('Puntos ordenados por distancia', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel(f'Distancia al {k_value}-ésimo vecino', fontsize=11, fontweight='bold')
    axes[idx].set_title(f'k = {k_value}\n{nombre_regla}', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].legend(loc='upper left', fontsize=8)

    # Zoom en la región del codo
    if eps_optimo < np.max(k_distances_sorted) * 0.5:
        axes[idx].set_ylim([0, np.percentile(k_distances_sorted, 99)])

    idx += 1

# Guardamos la figura con la función personalizada
save_fig('dbscan_knn_distance_optimal_eps', tight_layout=True, fig_extension="pdf", resolution=300)
plt.show()

# RESUMEN DE VALORES ÓPTIMOS
print("\n" + "="*80)
print(" RESUMEN DE VALORES EPS ÓPTIMOS POR MIN_SAMPLES")
print("="*80)
print(f"{'Min_Samples':<15} {'Eps (Knee)':<15} {'Eps (P90)':<15} {'Eps (P95)':<15} {'Regla'}")
print("-"*80)

for k_value, datos in resultados_eps.items():
    print(f"{k_value:<15} {datos['eps_knee']:<15.4f} {datos['eps_p90']:<15.4f} "
          f"{datos['eps_p95']:<15.4f} {datos['nombre'][:30]}")

# EVALUACIÓN DE LAS MEJORES COMBINACIONES
print("\n\n EVALUANDO CALIDAD DE CLUSTERING CON LAS COMBINACIONES ÓPTIMAS...\n")

evaluaciones = []

for k_value, datos in resultados_eps.items():
    # Probamos con el eps del knee y los percentiles
    for eps_tipo, eps_valor in [('Knee', datos['eps_knee']),
                                 ('P90', datos['eps_p90']),
                                 ('P95', datos['eps_p95'])]:

        # Aplicamos DBSCAN
        db_temp = DBSCAN(metric="precomputed", eps=eps_valor, min_samples=k_value)
        labels = db_temp.fit_predict(dist_gower)

        # Métricas
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        pct_noise = (n_noise / len(labels)) * 100

        # Calculamos silhouette si es posible
        silhouette = None
        if n_clusters >= 2 and n_noise < len(labels):
            mask = labels != -1
            if sum(mask) > 10:
                try:
                    silhouette = silhouette_score(dist_gower[mask][:, mask],
                                                   labels[mask],
                                                   metric='precomputed')
                except:
                    silhouette = None

        evaluaciones.append({
            'min_samples': k_value,
            'eps': eps_valor,
            'eps_tipo': eps_tipo,
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'pct_noise': pct_noise,
            'silhouette': silhouette,
            'regla': datos['nombre']
        })

# Convertimos a DataFrame y mostramos resultados
import pandas as pd
df_eval = pd.DataFrame(evaluaciones)

# Filtramos solo resultados válidos (con clusters y no demasiado ruido)
df_eval_validos = df_eval[(df_eval['n_clusters'] >= 2) & (df_eval['pct_noise'] < 50)].copy()

if len(df_eval_validos) > 0:
    # Ordenamos por menor ruido y luego por mayor silhouette
    df_eval_validos = df_eval_validos.sort_values(
        by=['pct_noise', 'silhouette'],
        ascending=[True, False]
    )

    print(" TOP 10 MEJORES CONFIGURACIONES:")
    print("="*100)
    print(df_eval_validos[['min_samples', 'eps', 'eps_tipo', 'n_clusters', 'pct_noise', 'silhouette']].head(10).to_string(index=False))

    # Mejor configuración
    mejor = df_eval_validos.iloc[0]

    print("\n" + "="*80)
    print(" CONFIGURACIÓN ÓPTIMA RECOMENDADA")
    print("="*80)
    print(f"\n PARÁMETROS:")
    print(f"   min_samples = {int(mejor['min_samples'])}")
    print(f"   eps = {mejor['eps']:.4f} (método: {mejor['eps_tipo']})")
    print(f"\n RESULTADOS ESPERADOS:")
    print(f"   • Número de clusters: {int(mejor['n_clusters'])}")
    print(f"   • Puntos de ruido: {int(mejor['n_noise'])} ({mejor['pct_noise']:.2f}%)")
    if mejor['silhouette'] is not None:
        print(f"   • Silhouette Score: {mejor['silhouette']:.4f}")
    print(f"   • Basado en: {mejor['regla']}")
print("\n" + "="*80)

In [ ]:
# Ajustamos el modelo DBSCAN usando la matriz de Gower
db = DBSCAN(metric="precomputed", eps=0.0605, min_samples=93) #eps = radio de la vecindad, min_samples indica cuantos punto tiene que tener como minimo un clouster
#esto despues se mejorara para encontrar el mejor eps
db.fit(dist_gower)

# Guardamos los clusters en un nuevo DataFrame
df_gow['cluster'] = db.labels_

In [ ]:
# Verificamos resultados
df_gow['cluster'].unique()

In [ ]:
print(f"Clusters encontrados: {len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)}")
print(f"Puntos de ruido: {list(db.labels_).count(-1)}")

Apreciamos que el algoritmo detecta 220 puntos de ruido, en proporcion respecto a mis 20,000 datos muestrales equivale al 1.1% lo cual esta en el rango aceptable. Visualizaremos ahora los clousters junto con el ruido

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
# --- OPCIÓN 1: PCA para exploracion ---
#pca = PCA(n_components=2, random_state=42)
#coords_pca = pca.fit_transform(dist_gower)

# --- OPCIÓN 2: t-SNE para grafico final ---
tsne = TSNE(n_components=2, metric="precomputed", random_state=42, perplexity=30, init="random")
coords_tsne = tsne.fit_transform(dist_gower)

# Elegimos una opción para graficar:
coords = coords_tsne #coords_pca  # o coords_tsne si usas t-SNE

# Agregamos coordenadas al DataFrame
df_gow['x'] = coords[:, 0]
df_gow['y'] = coords[:, 1]

# --- Graficamos ---
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_gow,
    x='x', y='y',
    hue='cluster',
    palette='tab10',  # paleta de colores
    s=15,
    alpha=0.8,
    edgecolor=None
)
plt.title('Visualización de Clusters DBSCAN (distancia de Gower)')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
save_fig("clusters_dbscan_gower", fig_extension="pdf", resolution=300)
plt.show()
